# LifeSnaps Preprocessing Experiments

#### Import de llibreries necessaries

In [1]:
import warnings
warnings.filterwarnings('ignore')

# Data handling and visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Imbalanced data pipeline
from imblearn.pipeline import Pipeline as ImbPipeline

# Core utilities
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.inspection import permutation_importance

# Model definitions fora dels defints
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


############## IMPORTS DEL NOSTRE PAQUET ###################
# Obtenim les funcions per entrenar i evaluar els models i registrar les mètriques
from ai_health_assistant.utils.train_helpers import train_models, append_results, plot_learning_curve, mat_confusio, update_experiments_file

# Obtenim els classificadors i els seus parametres
from ai_health_assistant.utils.model_config import get_classifier_config, PARAM_GRIDS, CLASSIFIERS, BALANCING_METHODS

# Obtenim el target, features el la construcció del preprocessador
from ai_health_assistant.utils.prep_helpers import TARGET, build_preprocessor, FEATURES


# Configuració de pandas
pd.set_option('display.max_columns', None)

# Carrega de dades, netes i amb fe aplicat
df_train = pd.read_csv('../data/df_engineered_train.csv')
df_test = pd.read_csv('../data/df_engineered_test.csv')
    
print(f"Shape: {df_train.shape}")
print(f"Shape: {df_test.shape}")


Shape: (1832, 38)
Shape: (458, 38)


## Lectura de dades i split de train / test

### Train / Test Split

In [2]:
# Fem l'split de les dades, separant les features i el target
X_train = df_train.drop(columns=[TARGET])
y_train = df_train[TARGET]

X_test = df_test.drop(columns=[TARGET])
y_test = df_test[TARGET]

print(f"\nTrain shape: {X_train.shape}")
print(f"Test shape: {X_test.shape}")
print('\n','--'*50)
print(f"\nDistribució train:\n{y_train.value_counts(normalize=True)}")
print(f"\nDistribució test:\n{y_test.value_counts(normalize=True)}")

numerical_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=['number']).columns.tolist()

print(f"\nCol. numeriques ({len(numerical_features)}): \n{numerical_features}")
print(f"Col. categoriques ({len(categorical_features)}): \n{categorical_features}")



Train shape: (1832, 37)
Test shape: (458, 37)

 ----------------------------------------------------------------------------------------------------

Distribució train:
TIRED
0.0    0.615721
1.0    0.384279
Name: proportion, dtype: float64

Distribució test:
TIRED
0.0    0.615721
1.0    0.384279
Name: proportion, dtype: float64

Col. numeriques (35): 
['bmi', 'calories', 'steps', 'lightly_active_minutes', 'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes', 'resting_hr', 'minutes_below_default_zone_1', 'minutes_in_default_zone_1', 'minutes_in_default_zone_2', 'minutes_in_default_zone_3', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency', 'sleep_deep_ratio', 'sleep_light_ratio', 'sleep_rem_ratio', 'sleep_wake_ratio', 'daily_temperature_variation', 'rmssd', 'spo2', 'full_sleep_breathing_rate', 'wake_after_sleep_pct', 'steps_norm_cal', 'deep_sleep_score', 'active_sedentary_ratio', 'sleep_activity_balance', 'bmi_hr_interaction', 'sleep_quality_index', 'hr_zone_variab

### Definim el preprocessador

In [3]:
preprocessor = build_preprocessor(df_train, FEATURES)

## ENTRENAMENT DEL MODEL BASE

In [4]:
display_cols = ['Experiment', 'Train F1 (1)', 'Train F1 (macro global)','Train Accuracy', 'Test Recall (1)', 'Test Precision (1)', 'Test F1 (1)', 'Test F1 (macro global)','Test Accuracy']

# --------------------------------------------------------------
# Filtre de selecció dels models a entrenar
PROVA_MODELS = ["BalancedRandomForest", "LGBM"]
# Selecció del mètode de balanceig
balance_name = "SMOTETomek"
# --------------------------------------------------------------

# Filtre per no haver de entrenar tots el models
CLASSIFIERS_FILTER = {k: v for k, v in CLASSIFIERS.items() if k in PROVA_MODELS}
balance_method = BALANCING_METHODS[balance_name]

### Regressió Logistica

In [6]:

reg_results = []
reg_models = {}

reg_pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LogisticRegression(
        max_iter=2000,            # convergència assegurada
        class_weight="balanced", # tracta l’imbalance de la classe 1
        solver="lbfgs",          # ràpid i estable per datasets petits/mitjans
    ))
])

reg_param_grid = {
    "classifier__C": [0.001, 0.01, 0.1, 1, 10, 100, 200, 500, 1000]
}

best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
        X_train, 
        y_train,
        X_test,
        y_test, 
        reg_pipeline, 
        reg_param_grid,
        search_type='grid',
    )

reg_results_df = append_results(
    reg_results,
    "LogisticRegression",
    train_report,
    test_report,
    best_params,
    best_score,
    experiment="EntrenamentBasic"
)

# update_experiments_file(reg_results_df)
display(reg_results_df[display_cols])
update_experiments_file(reg_results_df)

plt.show()

Entrenant model...

Train F1 (1): 0.5156 | Test F1 (1): 0.4800 | Train Acc: 0.5939 | Test Acc: 0.5742
              precision    recall  f1-score   support

         0.0     0.6680    0.6135    0.6396       282
         1.0     0.4523    0.5114    0.4800       176

    accuracy                         0.5742       458
   macro avg     0.5601    0.5624    0.5598       458
weighted avg     0.5851    0.5742    0.5782       458



,Experiment,Train F1 (1),Train F1 (macro global),Train Accuracy,Test Recall (1),Test Precision (1),Test F1 (1),Test F1 (macro global),Test Accuracy
0,LogisticRegression_EntrenamentBasic,0.51562,0.583,0.59389,0.51136,0.45226,0.48,0.55978,0.57424



Métriques guardades a ../results/02_experiments/experiments.csv



In [7]:
base_results = []

for model, classifier in CLASSIFIERS_FILTER.items():

    if model == "BalancedRandomForest":
        pipeline = ImbPipeline([
            ("preprocessor", preprocessor),
            ("classifier", classifier)
        ])
    else:
        pipeline = ImbPipeline([
            ("preprocessor", preprocessor),
            ("balance", balance_method),
            ("classifier", classifier)
        ])

    print(f"\n==== {model} ====")
    best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
        X_train,
        y_train,
        X_test,
        y_test, 
        pipeline,
        PARAM_GRIDS[model]
    )

    base_results_df = append_results(
    base_results,
    model,
    train_report,
    test_report,
    best_params,
    best_score,
    experiment="Entrenament basic"
)


==== BalancedRandomForest ====
Entrenant model...

Train F1 (1): 0.7136 | Test F1 (1): 0.5793 | Train Acc: 0.6960 | Test Acc: 0.5306
              precision    recall  f1-score   support

         0.0     0.7724    0.3369    0.4691       282
         1.0     0.4418    0.8409    0.5793       176

    accuracy                         0.5306       458
   macro avg     0.6071    0.5889    0.5242       458
weighted avg     0.6453    0.5306    0.5115       458


==== LGBM ====
Entrenant model...

Train F1 (1): 0.8812 | Test F1 (1): 0.5736 | Train Acc: 0.9023 | Test Acc: 0.6266
              precision    recall  f1-score   support

         0.0     0.7382    0.6099    0.6680       282
         1.0     0.5111    0.6534    0.5736       176

    accuracy                         0.6266       458
   macro avg     0.6247    0.6317    0.6208       458
weighted avg     0.6509    0.6266    0.6317       458



In [9]:
update_experiments_file(base_results_df)


Métriques guardades a ../results/02_experiments/experiments.csv



In [17]:
print(base_results_df[base_results_df['Model'] == "BalancedRandomForest"]['Best Params'].values[0])

{'classifier__n_estimators': 1163, 'classifier__min_samples_split': 5, 'classifier__min_samples_leaf': 3, 'classifier__max_features': 'log2', 'classifier__max_depth': 8, 'classifier__class_weight': 'balanced'}


In [18]:
# El model que vulguem inspecciona
model_name = "LGBM" # RandomForest, MLP, GradientBoosting, SVM
csf = base_models[model_name]


# Visualització del train
y_train_pred = csf.predict(X_train)

mat_confusio(
    f'Entrenament basic {model_name} (Train)',
    y_train,
    y_train_pred
)

# Visualització del test
y_test_pred = csf.predict(X_test)

mat_confusio(
    f"Entrenament basic {model_name} (Test)",
    y_test,
    y_test_pred
)

In [23]:
# Observem la corba d'aprenentatge:

plot_learning_curve(
    model_name,
    base_models[model_name],
    X_train,
    y_train
)

KeyboardInterrupt: 

## EXPERIMENT 1: Importancia de les caracteristiques

Entrenem RandomForest per indentificar les caracteristiques més importants (10-15), posteriorment entrenem els models utilitzant aquestes 10-15 característiques, per veure si augmenta el rendiment del model. Proavarem tambe amb permutation importances.

### Feature Importance

La Gini importance d’una feature és: La suma de totes les reduccions d’impuresa (Gini) que ha causat al llarg de tots els arbres i de totes les seves aparicions.

In [ ]:
# Param grid & pipeline bàsic de random forest
rf_name = "BalancedRandomForest"

pipeline = ImbPipeline([
    ("preprocessor", preprocessor),
    ("classifier", CLASSIFIERS[rf_name])
])

# Entrenament del model
best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
    X_train, 
    y_train,
    X_test,
    y_test, 
    pipeline, 
    PARAM_GRIDS[rf_name],
    search_type='grid'
)
preprocessor = best_est.named_steps['preprocessor']

if hasattr(preprocessor, 'get_feature_names_out'):
    feature_names = preprocessor.get_feature_names_out()
else:
    # If using older scikit-learn, you might need to handle this differently
    numeric_features = X_train.select_dtypes(include=['number']).columns
    categorical_features = X_train.select_dtypes(exclude=['number']).columns
    
    numeric_transformer = preprocessor.named_transformers_['num']
    if hasattr(numeric_transformer, 'get_feature_names_out'):
        numeric_feature_names = numeric_transformer.get_feature_names_out(numeric_features)
    else:
        numeric_feature_names = numeric_features
        
    categorical_transformer = preprocessor.named_transformers_['cat']
    if hasattr(categorical_transformer, 'get_feature_names_out'):
        categorical_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
    else:
        # For one-hot encoded features
        ohe = categorical_transformer.named_steps['onehot']
        categorical_feature_names = ohe.get_feature_names_out(categorical_features)
    
    feature_names = list(numeric_feature_names) + list(categorical_feature_names)

# Now create the Series with the correct feature names
importances_raw = pd.Series(
    best_est.named_steps["classifier"].feature_importances_,
    index=feature_names
)

def base_name(feat):
    if feat.startswith("cat__") and "_" in feat[6:]:
        return feat.rsplit("_", 1)[0]
    return feat

agg_importances = (
    importances_raw.groupby(base_name).sum().sort_values(ascending=False)
)

# Top-10 i Top-15 importàncies

top10 = agg_importances.head(10).index.tolist()
top15 = agg_importances.head(15).index.tolist()
print("Top-10 features:", top10)
print("Top-15 features:", top15)

# Visualització de les Top-15 importàncies
plt.figure(figsize=(10, 6))
plt.bar(top15, agg_importances.head(15).values)
plt.xticks(rotation=45, ha="right")
plt.title("Top-15 Gini Importances")
plt.xlabel("Feature")
plt.ylabel("Importància (Gini)")
plt.tight_layout()
plt.show()



Train F1 (1): 0.7136 | Test F1 (1): 0.5793 | Train Acc: 0.6960 | Test Acc: 0.5306
              precision    recall  f1-score   support

         0.0     0.7724    0.3369    0.4691       282
         1.0     0.4418    0.8409    0.5793       176

    accuracy                         0.5306       458
   macro avg     0.6071    0.5889    0.5242       458
weighted avg     0.6453    0.5306    0.5115       458

Top-10 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi', 'num__resting_hr', 'num__steps_norm_cal', 'num__daily_temperature_variation', 'num__recovery_factor', 'num__hr_zone_variability', 'num__lightly_active_minutes', 'num__minutesAsleep']
Top-15 features: ['num__calories', 'num__bmi_hr_interaction', 'num__bmi', 'num__resting_hr', 'num__steps_norm_cal', 'num__daily_temperature_variation', 'num__recovery_factor', 'num__hr_zone_variability', 'num__lightly_active_minutes', 'num__minutesAsleep', 'num__active_to_total_ratio', 'num__sedentary_minutes', 'num__minutes_below_

### Rentrenament de models 10-15 millors features

In [28]:
# Definim els dos conjunts de features
feature_sets = {
    "Top10": top10,   # llista de 10 noms de feature “base”
    "Top15": top15    # llista de 15 noms de feature “base”
}

# Reentrenament i avaluació per a cada subset
fi_results = []
fi_models = {}

numerical_features = X_train.select_dtypes(include=['number']).columns.tolist()
categorical_features = X_train.select_dtypes(exclude=['number']).columns.tolist()

preprocessor = build_preprocessor(X_train)
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

# Get the feature names after transformation
if hasattr(preprocessor, 'get_feature_names_out'):
    feature_names = preprocessor.get_feature_names_out()
else:
    # For older scikit-learn versions
    numeric_features = X_train.select_dtypes(include=['number']).columns
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns
    numeric_feature_names = list(numeric_features)
    categorical_feature_names = list(categorical_features)
    feature_names = numeric_feature_names + categorical_feature_names

# Convert to DataFrame with proper column names
X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)


for label, feats in feature_sets.items():
    print(f"\nEntrenament models amb {label}")
    num_feats = [f for f in feats if f in numerical_features]
    cat_feats = [f for f in feats if f in categorical_features]

    for model, classifier in CLASSIFIERS_FILTER.items():

        print(f'==== {model} ====')
        pipe = ImbPipeline([
            ("balance", balance_method),
            ("classifier", classifier)
        ])


        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models(
            X_train_transformed[feats],
            y_train,
            X_test_transformed[feats],
            y_test,
            pipe,
            PARAM_GRIDS[model]
        )

        fi_models[f"{model}_{label}"] = best_est

        append_results(
            fi_results,
            model,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=f"FI_{label}"
        )


Entrenament models amb Top10
==== BalancedRandomForest ====

Train F1 (1): 0.7546 | Test F1 (1): 0.4888 | Train Acc: 0.8051 | Test Acc: 0.6026
              precision    recall  f1-score   support

         0.0     0.6799    0.6702    0.6750       282
         1.0     0.4833    0.4943    0.4888       176

    accuracy                         0.6026       458
   macro avg     0.5816    0.5823    0.5819       458
weighted avg     0.6043    0.6026    0.6034       458

==== LGBM ====

Train F1 (1): 0.8374 | Test F1 (1): 0.5425 | Train Acc: 0.8597 | Test Acc: 0.5764
              precision    recall  f1-score   support

         0.0     0.7095    0.5284    0.6057       282
         1.0     0.4637    0.6534    0.5425       176

    accuracy                         0.5764       458
   macro avg     0.5866    0.5909    0.5741       458
weighted avg     0.6151    0.5764    0.5814       458


Entrenament models amb Top15
==== BalancedRandomForest ====

Train F1 (1): 0.7810 | Test F1 (1): 0.4903

In [29]:
feature_importance_results_df = pd.DataFrame(fi_results)

display(feature_importance_results_df[display_cols].round(4))

,Target,Experiment,Model,Train F1 (1),Test F1 (1),Train F1 (macro global),Test F1 (macro global),Train Accuracy,Test Accuracy
0,TIRED,FI_Top10,BalancedRandomForest,0.7546,0.4888,0.7965,0.5819,0.8051,0.6026
1,TIRED,FI_Top10,LGBM,0.8374,0.5425,0.8570,0.5741,0.8597,0.5764
2,TIRED,FI_Top15,BalancedRandomForest,0.7810,0.4903,0.8171,0.5809,0.8242,0.6004
3,TIRED,FI_Top15,LGBM,0.8561,0.5318,0.8741,0.5632,0.8766,0.5655


### Permutation Importance

Per avaluar la importància de les característiques del model. Serveix per determinar quines característiques tenen més impacte en el rendiment del model.

In [35]:
from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# Train the random forest model
rf_name = 'RandomForest'
clf_rf, param_grid_rf = get_classifier_config(rf_name)

# Create a new pipeline that includes the preprocessor
pipe = ImbPipeline([
    ("preprocessor", preprocessor),
    ("balance", balance_method),
    ("classifier", clf_rf)
])

# Train the model
best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models( 
    X_train, 
    y_train,
    X_test,
    y_test, 
    pipe, 
    param_grid_rf,
    search_type='grid'
)

best_rf_model = best_est

# To calculate permutation importance, we need to:
# 1. Get the preprocessor and classifier from the pipeline
preprocessor = best_rf_model.named_steps['preprocessor']
classifier = best_rf_model.named_steps['classifier']

# 2. Transform the test data using the preprocessor
X_test_transformed = preprocessor.transform(X_test)

# 3. Get feature names after transformation
if hasattr(preprocessor, 'get_feature_names_out'):
    feature_names = preprocessor.get_feature_names_out()
else:
    # Fallback for older scikit-learn versions
    numeric_features = X_train.select_dtypes(include=['number']).columns
    categorical_features = X_train.select_dtypes(include=['object', 'category']).columns
    
    # Get numeric feature names
    numeric_transformer = preprocessor.named_transformers_['num']
    if hasattr(numeric_transformer, 'get_feature_names_out'):
        numeric_feature_names = numeric_transformer.get_feature_names_out(numeric_features)
    else:
        numeric_feature_names = list(numeric_features)
    
    # Get categorical feature names
    categorical_transformer = preprocessor.named_transformers_['cat']
    if hasattr(categorical_transformer, 'get_feature_names_out'):
        categorical_feature_names = categorical_transformer.get_feature_names_out(categorical_features)
    else:
        ohe = categorical_transformer.named_steps['onehot']
        categorical_feature_names = ohe.get_feature_names_out(categorical_features)
    
    feature_names = list(numeric_feature_names) + list(categorical_feature_names)

# 4. Create a new pipeline with just the classifier for permutation importance
# (since we've already preprocessed the data)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Create a simple pipeline with just the classifier
final_estimator = Pipeline([
    ('classifier', classifier)
])

# 5. Calculate permutation importance using the transformed data
result = permutation_importance(
    final_estimator,
    X_test_transformed, 
    y_test,
    n_repeats=200,
    n_jobs=-1,
    random_state=42
)

# 6. Create Series with feature names
perm_importances = pd.Series(
    result.importances_mean, 
    index=feature_names
).sort_values(ascending=False)

# Extract Top-10 and Top-15 features
perm_top_features = {
    10: perm_importances.head(10).index.tolist(),
    15: perm_importances.head(15).index.tolist()
}

# Display the top 15 features
print("\nTop-15 features (Permutation):")
display(perm_importances.head(15).to_frame("Importancia"))

# Plot the top 15 features
plt.figure(figsize=(12, 6))
top15 = perm_importances.head(15)
bars = plt.barh(top15.index, top15.values)
plt.title("Top-15 Permutation Importance", fontsize=14)
plt.xlabel("Mean Decrease in F1 Score", fontsize=12)
plt.gca().invert_yaxis()  # Most important features on top
plt.grid(axis='x', linestyle='--', alpha=0.6)

# Add value labels on the bars
for bar in bars:
    width = bar.get_width()
    plt.text(width + 0.001, bar.get_y() + bar.get_height()/2., 
             f'{width:.3f}', 
             ha='left', va='center')

plt.tight_layout()
plt.show()


Train F1 (1): 0.8037 | Test F1 (1): 0.4900 | Train Acc: 0.8499 | Test Acc: 0.6092
              precision    recall  f1-score   support

         0.0     0.6820    0.6844    0.6832       282
         1.0     0.4914    0.4886    0.4900       176

    accuracy                         0.6092       458
   macro avg     0.5867    0.5865    0.5866       458
weighted avg     0.6088    0.6092    0.6090       458


Top-15 features (Permutation):


,Importancia
num__bmi,0.031998
num__daily_temperature_variation,0.013526
cat__age_<30,0.008373
num__minutesAwake,0.006648
num__sleep_efficiency,0.005273
num__sleep_quality_index,0.004258
num__sleep_deep_ratio,0.003908
num__deep_sleep_score,0.003570
num__rmssd,0.003483
num__sleep_wake_ratio,0.002871


### Reentrenament Permutation importance

In [37]:
# First, let's see what features we actually have
print("Available columns in X_train:", X_train.columns.tolist())
print("\nTrying to select features:", sel_feats)

# The issue is that sel_feats contains transformed feature names (with num__ and cat__ prefixes)
# but X_train has the original feature names. We need to map between them.

# Create a mapping from transformed names to original names
# This depends on how your preprocessor is set up
original_features = X_train.columns.tolist()

# If you're using a ColumnTransformer, you can get the feature names like this:
if hasattr(preprocessor, 'get_feature_names_out'):
    transformed_features = preprocessor.get_feature_names_out()
    # Create a mapping from transformed to original names
    # This is a simple mapping - adjust based on your actual transformation
    feature_mapping = {}
    for orig in original_features:
        for trans in transformed_features:
            if orig in trans:  # Simple check - might need more sophisticated mapping
                feature_mapping[trans] = orig
                break
    
    # Now map the selected features back to original names
    original_sel_feats = []
    for feat in sel_feats:
        # Remove the prefix (num__ or cat__) to get the base name
        base_name = feat.split('__', 1)[-1] if '__' in feat else feat
        # Find the original feature that matches this base name
        for orig in original_features:
            if base_name in orig:
                original_sel_feats.append(orig)
                break
    original_sel_feats = list(set(original_sel_feats))  # Remove duplicates
    
    print("\nMapped features:", original_sel_feats)
    
    # Now use these original feature names for selection
    X_train_sel = X_train[original_sel_feats]
    X_test_sel = X_test[original_sel_feats]
    
    # Now train the models with the selected features
    for model, classifier in CLASSIFIERS_FILTER.items():
        pipe = ImbPipeline([
            ("preprocessor", preprocessor),
            ("balance", balance_method),
            ("classifier", classifier)
        ])

        best_est, y_train_pred, train_report, y_test_pred, test_report, best_params, best_score = train_models( 
            X_train_sel, 
            y_train,
            X_test_sel,
            y_test, 
            pipe, 
            PARAM_GRIDS[model]
        )

        perm_models[f'{model}_Top{k}'] = best_est

        append_results(
            perm_results,
            model,
            best_params,
            y_train,
            y_train_pred,
            y_test,
            y_test_pred,
            experiment=f"Perm_Top{k}"
        )
else:
    print("Could not get feature names from preprocessor. Please check your preprocessor setup.")
    # Fallback: Use the original features that match any of the selected feature names
    original_sel_feats = [f for f in original_features if any(sf.split('__')[-1] in f for sf in sel_feats)]
    print("Trying with features:", original_sel_feats)
    
    if original_sel_feats:
        X_train_sel = X_train[original_sel_feats]
        X_test_sel = X_test[original_sel_feats]
        
        # Rest of the training code as above
        # ...
    else:
        print("No matching features found. Please check your feature names.")

Available columns in X_train: ['age', 'gender', 'bmi', 'calories', 'steps', 'lightly_active_minutes', 'moderately_active_minutes', 'very_active_minutes', 'sedentary_minutes', 'resting_hr', 'minutes_below_default_zone_1', 'minutes_in_default_zone_1', 'minutes_in_default_zone_2', 'minutes_in_default_zone_3', 'minutesAsleep', 'minutesAwake', 'sleep_efficiency', 'sleep_deep_ratio', 'sleep_light_ratio', 'sleep_rem_ratio', 'sleep_wake_ratio', 'daily_temperature_variation', 'rmssd', 'spo2', 'full_sleep_breathing_rate', 'wake_after_sleep_pct', 'steps_norm_cal', 'deep_sleep_score', 'active_sedentary_ratio', 'sleep_activity_balance', 'bmi_hr_interaction', 'sleep_quality_index', 'hr_zone_variability', 'recovery_factor', 'sleep_eff_rmssd', 'active_to_rest_transition', 'active_to_total_ratio']

Trying to select features: ['num__bmi', 'num__daily_temperature_variation', 'cat__age_<30', 'num__minutesAwake', 'num__sleep_efficiency', 'num__sleep_quality_index', 'num__sleep_deep_ratio', 'num__deep_sleep

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "d:\OneDrive\Documentos\02. Formació\2. UNIVERSITAT\05. TFG\02. Code\AI-Health-Assistant\.venv\Lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
           ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'ca

In [29]:
perm_topk_results_df = pd.DataFrame(perm_results)
display(perm_topk_results_df[display_cols].round(4))

,Target,Experiment,Model,Train F1 (1),Test F1 (1),Train F1 (macro global),Test F1 (macro global),Train Accuracy,Test Accuracy
0,TIRED,Perm_Top10,BalancedRandomForest,0.7398,0.5893,0.7981,0.6850,0.8150,0.7140
1,TIRED,Perm_Top10,LGBM,0.9403,0.6054,0.9508,0.6690,0.9531,0.6812
2,TIRED,Perm_Top15,BalancedRandomForest,0.7985,0.6272,0.8388,0.7046,0.8488,0.7249
3,TIRED,Perm_Top15,LGBM,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878


## EXPERIMENT 2: PCA


Es realitza una anàlisi de components principals (PCA) per examinar com evolucionen els components més rellevants del conjunt de dades en termes de variància explicada acumulada, considerant els primers 5, 10, 15, 20 i 25 components

In [30]:
pca = PCA(random_state=42)
pca.fit(X_train)

# 3) Calcular la varianza explicada acumulada
explained_cumsum = pca.explained_variance_ratio_.cumsum()*100

# 4) Definir los puntos de interés y extraer sus valores
ks = [5, 10, 15, 20, 25]
cums = explained_cumsum[[k-1 for k in ks]]

# 5) Dibujar la curva completa y señalar los ks elegidos
plt.figure(figsize=(8, 4))
plt.plot(
    range(1, len(explained_cumsum) + 1),
    explained_cumsum,
)
plt.scatter(ks, cums)
plt.xlabel('Número de componentes')
plt.ylabel('Varianza explicada acumulada')
plt.title('Evolución de la varianza explicada según n_components')
plt.grid(True)
plt.tight_layout()
plt.show()

In [31]:
n_components_list = [5, 10, 15, 20, 25]
for k in n_components_list:
    # Ajusta PCA
    pca = PCA(n_components=k, random_state=42)
    pca.fit(X_train)

    # loadings: matriz (n_features, k)
    loadings = pca.components_.T

    # importancia = suma de cargas absolutes de cada feature en tots els components
    importance = np.sum(np.abs(loadings), axis=1)

    # crea DataFrame, ordena top-k
    df_imp = pd.DataFrame({
        'feature':    all_features,
        'importance': importance
    }).sort_values('importance', ascending=False).reset_index(drop=True)
    topk = df_imp.head(k)

    plt.figure()
    plt.barh(topk['feature'][::-1], topk['importance'][::-1])
    plt.xlabel('Importancia (suma de |carregues|)')
    plt.title(f'Top {k} features segons PCA')
    plt.tight_layout()
    plt.show()

Aquests gràfics mostren, per cada valor de *k* (5, 10, 15, 20 i 25 components), quines variables original s’aporten més a l’espai de la PCA i, per tant, expliquen més variància del conjunt de dades.

* **Components principals**: són noves variables creades com a combinacions lineals de les variables originals.
* **Càrregues (loadings)**: cada component té un coeficient per a cada variable; aquell coeficient indica quant “pesa” la variable en aquest eix.
* **Importància de la variable**: per a cada variable, sumem el valor absolut de les càrregues als primers *k* components. Una suma més alta vol dir que la variable contribueix de manera rellevant a la variació capturada per aquests *k* eixos.

Així podem veure quines són les *k* variables que més pesen ens diu quins atributs són més informatius (i quins, en canvi, aporten informació redundant).
Els tops *k* ajuden a identificar les característiques més representatives del dataset segons la PCA.


In [32]:
# Guarda resultats i models
pca_results = []
pca_models = {}


for k in n_components_list:
    print(f"\n-- PCA - {k} components --")

    for model, classifier in CLASSIFIERS_FILTER.items():
        # Pipeline amb preprocessor, SMOTE, PCA i classificador
        pipeline = ImbPipeline([
            ("balance",       balance_method),
            ("pca",          PCA(n_components=k, random_state=42)),
            ("classifier",   classifier)
        ])

        best_est, y_pred, report, best_params, best_score, best_params, best_score = train_models( 
        X_train, 
        y_train,
        X_test,
        y_test,
        pipeline, 
        PARAM_GRIDS[model],
        search_type="grid"
        )

        # Guarda el millor model
        pca_models[f'{model}_PCA{k}'] = best_est

        df_pca_results = append_results(
            pca_results,
            model,
            train_report,
            test_report,
            best_params,
            best_score,
            experiment=f"PCA_{k}",
        )



-- PCA - 5 components --

Train F1 (1): 0.7491 | Test F1 (1): 0.4985 | Train Acc: 0.8160 | Test Acc: 0.6354
              precision    recall  f1-score   support

         0.0     0.6910    0.7376    0.7136       282
         1.0     0.5287    0.4716    0.4985       176

    accuracy                         0.6354       458
   macro avg     0.6098    0.6046    0.6060       458
weighted avg     0.6286    0.6354    0.6309       458


Train F1 (1): 0.7073 | Test F1 (1): 0.5000 | Train Acc: 0.7118 | Test Acc: 0.4629
              precision    recall  f1-score   support

         0.0     0.6268    0.3156    0.4198       282
         1.0     0.3892    0.6989    0.5000       176

    accuracy                         0.4629       458
   macro avg     0.5080    0.5072    0.4599       458
weighted avg     0.5355    0.4629    0.4506       458


-- PCA - 10 components --

Train F1 (1): 0.8208 | Test F1 (1): 0.5784 | Train Acc: 0.8608 | Test Acc: 0.6594
              precision    recall  f1-score 

In [33]:
pca_results_df = pd.DataFrame(pca_results)
display(pca_results_df[display_cols].round(4))

,Target,Experiment,Model,Train F1 (1),Test F1 (1),Train F1 (macro global),Test F1 (macro global),Train Accuracy,Test Accuracy
0,TIRED,PCA_5,BalancedRandomForest,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
1,TIRED,PCA_5,LGBM,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
2,TIRED,PCA_10,BalancedRandomForest,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
3,TIRED,PCA_10,LGBM,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
4,TIRED,PCA_15,BalancedRandomForest,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
5,TIRED,PCA_15,LGBM,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
6,TIRED,PCA_20,BalancedRandomForest,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
7,TIRED,PCA_20,LGBM,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
8,TIRED,PCA_25,BalancedRandomForest,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878
9,TIRED,PCA_25,LGBM,0.9474,0.6207,0.9566,0.6777,0.9585,0.6878


## Anàlisi de resultats

In [34]:
# --- Results Compilation ---
all_results_dfs = [base_results_df]
if 'feature_importance_results_df' in locals() and not feature_importance_results_df.empty: 
    all_results_dfs.append(feature_importance_results_df)

if 'perm_topk_results_df' in locals() and not perm_topk_results_df.empty: 
    all_results_dfs.append(perm_topk_results_df)

if 'pca_results_df' in locals() and not pca_results_df.empty: 
    all_results_dfs.append(pca_results_df)

if len(all_results_dfs) > 1:
    combined_results_df = pd.concat(all_results_dfs, ignore_index=True)
    print("\nCombined results from all experiments.")
else:
    print("\nOnly baseline results available.")
    combined_results_df = base_results_df

final_cols = ["Target", "Experiment", "Model", "Test Accuracy","Test F1 (1)", "Test F1 (macro global)","Best Params"]
 
combined_results_df = combined_results_df[final_cols]

# --- Analysis ---
print("\n--- Overall Performance Analysis (Sorted by Test F1-Macro) ---")
combined_results_sorted = combined_results_df.sort_values(by=["Test Accuracy"], ascending=[False]).reset_index(drop=True)


print(f"\n--- Target: {TARGET} ---")
display(combined_results_sorted[final_cols].head().round(4))


Combined results from all experiments.

--- Overall Performance Analysis (Sorted by Test F1-Macro) ---

--- Target: TIRED ---


,Target,Experiment,Model,Test Accuracy,Test F1 (1),Test F1 (macro global),Best Params
0,TIRED,Entrenament basic,LGBM,0.7314,0.6516,0.7165,"{'classifier__subsample': 0.7590327755184709, ..."
1,TIRED,Entrenament basic,BalancedRandomForest,0.7249,0.6250,0.7039,"{'classifier__n_estimators': 1163, 'classifier..."
2,TIRED,Perm_Top15,BalancedRandomForest,0.7249,0.6272,0.7046,"{'classifier__n_estimators': 1163, 'classifier..."
3,TIRED,FI_Top15,BalancedRandomForest,0.7227,0.5968,0.6928,"{'classifier__n_estimators': 1163, 'classifier..."
4,TIRED,FI_Top10,BalancedRandomForest,0.7162,0.5667,0.6778,"{'classifier__n_estimators': 1163, 'classifier..."


In [35]:
tired_top5 = combined_results_sorted.head(5)

# Mostra la taula resum dels 5 millors
print(f"\nTop 5 models per a {TARGET} segons Test Accuracy:\n")
display(tired_top5[["Model","Test Accuracy","Test F1 (1)"]])

# Dibuixa la matriu de confusió de cada un
for model in tired_top5["Model"]:
    clf = base_models[model] 
    y_pred = clf.predict(X_test)  
    cm = confusion_matrix(y_test, y_pred, labels=[0,1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix — {TARGET} — {model}")
    plt.show()


Top 5 models per a TIRED segons Test Accuracy:



,Model,Test Accuracy,Test F1 (1)
0,LGBM,0.731441,0.651558
1,BalancedRandomForest,0.724891,0.625000
2,BalancedRandomForest,0.724891,0.627219
3,BalancedRandomForest,0.722707,0.596825
4,BalancedRandomForest,0.716157,0.566667
